# Setting boundaries on model parameters

In previous parts of the tutorial we've looked at simulation methods and performed some basic fitting.

In this notebook, we'll try to set some _boundaries_ on the parameter values we expect.
This will make our life easier in three ways:

1. By staying within a sensible region of parameter space, we can avoid numerical issues
2. By reducing the size of the search space, we can speed up optimisations and make them more robust
3. By sampling from within known boundaries we can derive random parameter sets to use as starting points for our search.

## Dealing with simulation issues in a ForwardModel

First, we'll show what can happen when you don't define boundaries.
We'll begin by defining a forward model:

In [9]:
import matplotlib.pyplot as plt
import myokit
import myokit.lib.hh
import numpy as np
import pints

class Model(pints.ForwardModel):
    """A forward model that runs simulations on step protocols."""
    
    def __init__(self, protocol):
        
        # Load a model, and isolate the HH ion current model part
        model = myokit.load_model('resources/beattie-2017-ikr-hh.mmt')
        parameters = ['ikr.p' + str(1 + i) for i in range(9)]        
        hh_model = myokit.lib.hh.HHModel.from_component(model.get('ikr'), parameters=parameters)
        
        # Create an analytical simulation
        self.sim = myokit.lib.hh.AnalyticalSimulation(hh_model, protocol)
        
        # Set the -80mV steady state as the default state
        self.sim.set_default_state(hh_model.steady_state(-80))
        
    def n_parameters(self):
        return 9
    
    def simulate(self, parameters, times):

        # Reset, apply parameters, and run
        self.sim.reset()
        self.sim.set_parameters(parameters)
        log = self.sim.run(times[-1] + 0.1, log_times=times)
        return log['ikr.IKr']    

In [10]:
# Load a protocol and create a foward model instance
protocol = myokit.load_protocol('resources/simplified-staircase.mmt')
model = Model(protocol)

# Define a parameter vector
parameters = np.array([3e-4, 0.07, 3e-5, 0.05, 0.09, 9e-3, 5e-3, 0.03, 0.2])

# Set up a problem
times = np.arange(0, 15400, 0.1)
values = model.evaluate(times) + np.random.normal(0, 0.015, len(times))

#

ValueError: The values in `state` must sum to 1.

## How to come up with parameter bounds

## Using boundaries in an optimisation

## Summary

In this part of the tutorial, we have

- Lalala

In the next part